# Style-Preserving Speech-to-Speech Translation Experiment

This notebook runs the experiment to determine the minimal duration of speaker embeddings required to effectively clone a speaker's voice across languages.

## 1. Setup Environment
Install necessary dependencies if running on Google Colab.

In [35]:
#only if needed, clear all files except experiment.ipynb
# This command will remove all files and folders in the current directory except "experiment.ipynb"
import os

for fname in os.listdir():
    if fname != "experiment.ipynb":
        if os.path.isdir(fname):
            import shutil
            shutil.rmtree(fname)
        else:
            os.remove(fname)







In [36]:
# Cell to refresh code from GitHub
import os

# Navigate to the repo directory
if os.path.exists("CS479-SpeakerEmbeddings"):
    os.chdir("CS479-SpeakerEmbeddings")
    !git pull
else:
    !git clone https://github.com/NathanAsayDong/CS479-SpeakerEmbeddings.git
    os.chdir("CS479-SpeakerEmbeddings")

# Optional: Reload modules if you've already imported them
import sys
import importlib

# List of your custom modules to reload
modules_to_reload = [
    "common_voice_dataset",
    "setup_experiment",
    "run_experiment",
    "asr_service",
    "translation_service",
    "tts_service",
    "embedding_service",
    "synthetic_data_service",
    "enums"
]

for module_name in modules_to_reload:
    if module_name in sys.modules:
        importlib.reload(sys.modules[module_name])
        print(f"Reloaded {module_name}")

Cloning into 'CS479-SpeakerEmbeddings'...
remote: Enumerating objects: 107, done.
remote: Counting objects: 100% (107/107), done.
remote: Compressing objects: 100% (75/75), done.
remote: Total 107 (delta 58), reused 79 (delta 30), pack-reused 0 (from 0)
Receiving objects: 100% (107/107), 624.24 KiB | 1.40 MiB/s, done.
Resolving deltas: 100% (58/58), done.
Reloaded common_voice_dataset
Reloaded setup_experiment
Reloaded run_experiment
Reloaded asr_service
Reloaded translation_service
Reloaded tts_service
Reloaded embedding_service
Reloaded synthetic_data_service
Reloaded enums


In [37]:

# %cd CS479-SpeakerEmbeddings
# !ls

In [38]:
!pip install torch transformers speechbrain soundfile librosa openai-whisper accelerate sentencepiece pydantic torchcodec datasets kagglehub[pandas-datasets]
!pip install sounddevice
!sudo apt-get install libportaudio2

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libportaudio2 is already the newest version (19.6.0-1.1).
0 upgraded, 0 newly installed, 0 to remove and 41 not upgraded.


## 2. Import Modules
Import the experiment setup and runner classes.

In [39]:
import os
import sys

# Add current directory to path if needed
sys.path.append(os.getcwd())

from enums import Language
from setup_experiment import ExperimentSetup
from run_experiment import ExperimentRunner

## 3. Configure Experiment
Define the parameters for the experiment: source/target languages and reference durations to test.

In [40]:
SOURCE_LANG = Language.ENGLISH
TARGET_LANG = Language.SPANISH
DURATIONS = [1.0, 2.0, 3.0, 4.0, 5.0]
NUM_SAMPLES_PER_DURATION = 5 # Number of unique speakers to test
SEED = 42

## 4. Prepare Data
This step:
1. Downloads/Loads Common Voice dataset via KaggleHub.
2. Selects `NUM_SPEAKERS` with sufficient data.
3. Creates concatenated reference audio files for each duration.
4. Generates a manifest for the experiment run.

In [41]:
!ls

asr_service.py		 peoples_speech_dataset.py  setup_experiment.py
common_voice_dataset.py  ProjectOutline.pdf	    synthetic_data_service.py
embedding_service.py	 __pycache__		    tmp_model
enums.py		 readMe			    translation_service.py
experiment.ipynb	 requirements.txt	    tts_service.py
libri_speech_dataset.py  run_experiment.py
main.py			 Samples


In [42]:
setup = ExperimentSetup(
    source_language=SOURCE_LANG,
    target_language=TARGET_LANG,
    reference_durations=DURATIONS,
    seed=SEED
)

# Prepare the manifest
manifest = setup.prepare_data(num_samples_per_duration=NUM_SAMPLES_PER_DURATION)

print(f"Manifest ready with {len(manifest)} speakers.")
print("Sample Item:", manifest[0] if manifest else "No data")

Preparing experiment data: 5 samples for each of [1.0, 2.0, 3.0, 4.0, 5.0]s durations...
Loading Common Voice dataset for language 'en'...
Using Colab cache for faster access to the 'common-voice' dataset.
Dataset path: /kaggle/input/common-voice
Searching for language 'en' in /kaggle/input/common-voice
Found flattened dataset structure at /kaggle/input/common-voice
Loaded 4076 records for en/dev
Columns: ['filename', 'text', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'duration', 'path', 'sentence']
Manifest ready with 25 speakers.
Sample Item: {'sample_id': 'sample_1', 'test_input_path': '/kaggle/input/common-voice/cv-valid-dev/cv-valid-dev/sample-001749.mp3', 'test_input_text': 'and in that way the months passed', 'reference_path': 'experiment_data/sample_1/ref_1s.wav', 'target_duration': 1.0}


## 5. Run Experiment
Execute the pipeline for each speaker and duration:
1. Extract ground truth embedding (original speaker).
2. Translate source text to Spanish.
3. Synthesize Spanish speech using the reference audio (5s, 10s, etc.) for style.
4. Compute Cosine Similarity between ground truth and output embeddings.

In [43]:
runner = ExperimentRunner()
runner.run(manifest)

/usr/local/lib/python3.12/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
INFO:speechbrain.utils.fetching:Fetch hyperparams.yaml: Fetching from HuggingFace Hub 'speechbrain/spkrec-xvect-voxceleb' if not cached
DEBUG:speechbrain.utils.fetching:Fetch: Local file found, creating symlink '/root/.cache/huggingface/hub/models--speechbrain--spkrec-xvect-voxceleb/snapshots/56895a2df401be4150a159f3a1c653f00051d477/hyperparams.yaml' -> '/content/CS479-SpeakerEmbeddings/CS479-SpeakerEmbeddings/CS479-SpeakerEmbeddings/CS479-SpeakerEmbeddings/tmp_model/hyperparams.yaml'
INFO:speechbrain.utils.fetching:Fetch custom.py: Fetching from HuggingFace Hub 'speechbrain/spkrec-xvect-voxceleb' if not cached
DEBUG:speechbrain.utils.parameter_transfer:Collecting files (or symlinks) for pretraining in tmp_model.
INFO:speechbrain.utils.fetching:Fetch embedding_model.ckpt: Fetching from Hu

Starting experiment run...
Running: Sample sample_1 | Ref Duration: 1.0s
  -> Similarity: 0.8529 | Text match len: 27/23
Running: Sample sample_1 | Ref Duration: 2.0s
  -> Similarity: 0.9470 | Text match len: 20/23
Running: Sample sample_1 | Ref Duration: 3.0s
  -> Similarity: 0.9483 | Text match len: 17/23
Running: Sample sample_1 | Ref Duration: 4.0s
  -> Similarity: 0.9631 | Text match len: 25/23
Running: Sample sample_1 | Ref Duration: 5.0s
  -> Similarity: 0.9632 | Text match len: 25/23
Running: Sample sample_2 | Ref Duration: 1.0s
  -> Similarity: 0.9321 | Text match len: 58/52
Running: Sample sample_2 | Ref Duration: 2.0s
  -> Similarity: 0.9294 | Text match len: 51/52
Running: Sample sample_2 | Ref Duration: 3.0s
  -> Similarity: 0.9367 | Text match len: 56/52
Running: Sample sample_2 | Ref Duration: 4.0s
  -> Similarity: 0.9354 | Text match len: 53/52
Running: Sample sample_2 | Ref Duration: 5.0s
  -> Similarity: 0.9350 | Text match len: 54/52
Running: Sample sample_3 | Ref Du

## 6. Analyze Results
Save and inspect the results.

In [44]:
runner.save_results("experiment_results.csv")

import pandas as pd
results_df = pd.read_csv("experiment_results.csv")

# Display average similarity score per duration
print("\nAverage Similarity Scores by Duration:")
print(results_df.groupby("duration")["similarity_score"].mean())

results_df.head(10)

Results saved to experiment_results.csv

Average Similarity Scores by Duration:
duration
1.0    0.875905
2.0    0.938237
3.0    0.942902
4.0    0.944907
5.0    0.945699
Name: similarity_score, dtype: float64


,sample_id,duration,similarity_score,original_text,target_text,transcribed_text,output_path
0,sample_1,1.0,0.852913,and in that way the months passed,y así pasaron los meses,La maillana da un poco de풋.,experiment_data/sample_1/out_sample_1_1.0s.wav
1,sample_1,2.0,0.947049,and in that way the months passed,y así pasaron los meses,y no las cosas mudas,experiment_data/sample_1/out_sample_1_2.0s.wav
2,sample_1,3.0,0.948252,and in that way the months passed,y así pasaron los meses,Pazerun los Meses,experiment_data/sample_1/out_sample_1_3.0s.wav
3,sample_1,4.0,0.963131,and in that way the months passed,y así pasaron los meses,A.S. Pesar and Lost Meces,experiment_data/sample_1/out_sample_1_4.0s.wav
4,sample_1,5.0,0.963196,and in that way the months passed,y así pasaron los meses,I.S. Pesar and Lost Meces,experiment_data/sample_1/out_sample_1_5.0s.wav
5,sample_2,1.0,0.932076,about time one of you lunkheads said it,"Ya era hora de que lo dijera uno de ustedes, t...",Ya era horror de koos lo dejaba a no dar este ...,experiment_data/sample_2/out_sample_2_1.0s.wav
6,sample_2,2.0,0.929379,about time one of you lunkheads said it,"Ya era hora de que lo dijera uno de ustedes, t...","Ya era hora de clío de draa, a no de estiz, ta...",experiment_data/sample_2/out_sample_2_2.0s.wav
7,sample_2,3.0,0.936690,about time one of you lunkheads said it,"Ya era hora de que lo dijera uno de ustedes, t...",Ya era hora de que lo digeron odio a las estad...,experiment_data/sample_2/out_sample_2_3.0s.wav
8,sample_2,4.0,0.935405,about time one of you lunkheads said it,"Ya era hora de que lo dijera uno de ustedes, t...",Ya era hora de culo de Dijera a no de Estid's ...,experiment_data/sample_2/out_sample_2_4.0s.wav
9,sample_2,5.0,0.934952,about time one of you lunkheads said it,"Ya era hora de que lo dijera uno de ustedes, t...","Ya era hora de culo, didgera, a no de a steeds...",experiment_data/sample_2/out_sample_2_5.0s.wav


## 7. Real-World Demo
Record your own voice, translate it, and check the similarity score.
NOTE: This requires a microphone. If running on a remote Colab kernel without audio forwarding, this might not work directly.

In [ ]:
# Real-World Test
from asr_service import ASRService
from translation_service import TranslationService
from tts_service import TTSService
from embedding_service import EmbeddingService
from enums import Language
import torch.nn.functional as F

def run_live_demo(duration=10):
    print(f"--- Live Demo (Enrollment: {duration}s) ---")
    
    # Initialize services
    asr = ASRService()
    translator = TranslationService()
    tts = TTSService()
    embedder = EmbeddingService()
    
    # 1. Record Input
    try:
        # Step 1: Listen and Transcribe
        print("\nRecording... (speak now)")
        english_text, source_audio_path = asr.listen_transcribe(duration=5)
        print(f"Transcribed: {english_text}")
        
        # Step 2: Record Enrollment (Reference)
        # Ideally we use the same audio if it's long enough, but let's record a dedicated style clip
        input("Press Enter to record style enrollment (speak clearly)...")
        ref_path = asr.record_audio(duration=duration, file_path="demo_ref.wav")
        
        # # Simpler: Just use the input audio itself as reference (Zero-Shot on self)
        # ref_path = source_audio_path

        # 2. Translate
        spanish_text = translator.translate(english_text, target_language=Language.SPANISH)
        print(f"Translated: {spanish_text}")

        # 3. Synthesize
        output_path = "demo_output.wav"
        tts.synthesize(spanish_text, output_path, ref_path)
        print(f"Synthesized audio saved to {output_path}")

        # 4. Evaluate Similarity
        gt_embedding = embedder.extract_embedding(ref_path)
        out_embedding = embedder.extract_embedding(output_path)
        
        if gt_embedding.dim() == 1: gt_embedding = gt_embedding.unsqueeze(0)
        if out_embedding.dim() == 1: out_embedding = out_embedding.unsqueeze(0)
            
        score = F.cosine_similarity(gt_embedding, out_embedding).item()
        print(f"Speaker Similarity Score: {score:.4f}")
        
        # Playback (if in Colab/Jupyter)
        from IPython.display import Audio, display
        print("Original:")
        display(Audio(source_audio_path))
        print("Synthesized (Spanish):")
        display(Audio(output_path))
        
    except Exception as e:
        print(f"Error during demo: {e}")
        print("Note: Microphone recording might fail on remote servers without audio forwarding.")

run_live_demo(duration=10)
# To run this on Colab, you'd typically need a Javascript helper to record audio from the browser,
# as 'sounddevice' tries to access the server's mic (which doesn't exist).
# For now, this code works if running locally.

--- Live Demo (Enrollment: 10s) ---


/Users/nathandong/Desktop/School/CS479_Project/venv/lib/python3.13/site-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
/Users/nathandong/Desktop/School/CS479_Project/venv/lib/python3.13/site-packages/speechbrain/utils/autocast.py:188: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  wrapped_fwd = torch.cuda.amp.custom_fwd(fwd, cast_inputs=cast_inputs)



Recording... (speak now)
Listening for 5 seconds...


/Users/nathandong/Desktop/School/CS479_Project/venv/lib/python3.13/site-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Transcribed: Hello, my name is Nathan and I want to run a test really quickly.
Translated: Hola, mi nombre es Nathan y quiero hacer una prueba muy rápido.
Synthesized audio saved to demo_output.wav
Speaker Similarity Score: 0.9721
Original:


Synthesized (Spanish):
